In [23]:
!pip install -q bitsandbytes datas accelerate loralib sentencepiece sentence-transformers evaluate
!pip install tensorboardX
!pip install -q git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/peft.git

In [ ]:
import os
os.kill(os.getpid(), 9)

In [36]:
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training
import datas
from datas import load_dataset
from sklearn import preprocessing
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn

In [3]:
tag_list = open(r'data/dictionaries/tag_list.txt').read().splitlines()

,tag_string_general,has_visible_children,has_large,tag_count_meta,bit_flags,has_active_children,preview_file_url,image_width,updated_at,tag_string_artist,...,last_commented_at,last_comment_bumped_at,parent_id,approver_id,pixiv_id,last_noted_at,filtered_tag_string,prompt_format_id,prompt_string,caption_string
0,1girl ;p ahoge bed_sheet blurry blurry_backgro...,False,1.0,1,2,False,https://cdn.donmai.us/preview/a4/77/a4773ba245...,1100,2021-09-29 10:33:58.201,tagame_(tagamecat),...,NaT,NaT,NaN,NaN,66118571.0,NaT,1girl ;p ahoge bed_sheet blurry blurry_backgro...,0,Reply with an eloquent caption based on all th...,"""Tagame's masterful artistry captures Kanbe Ko..."
1,1girl apron barefoot blonde_hair blush breasts...,False,0.0,4,0,False,https://cdn.donmai.us/preview/e0/85/e085faf380...,800,2021-11-07 13:44:46.477,yukataro,...,2013-06-06 00:46:40.388,2013-06-06 00:46:40.388,NaN,NaN,NaN,2013-06-03 18:45:36.358000128,1girl apron barefoot blonde_hair blush breasts...,1,Reply with a concise description of an artwork...,"The artwork is a sketch of Yakumo Ran, a chara..."
2,1girl animal_ears bandaid bandaid_on_head biki...,False,1.0,2,2,False,https://cdn.donmai.us/preview/54/82/5482e866cc...,3314,2021-11-08 13:31:08.499,elsynien,...,NaT,NaT,4899853.0,131474.0,NaN,NaT,1girl animal_ears bandaid bikini breasts brown...,0,Reply with an eloquent caption based on all th...,Elsynien's exquisite artistry brings to life t...
3,1girl :o arms_up bangs blue_dress blue_eyes bl...,False,1.0,2,2,False,https://cdn.donmai.us/preview/30/30/3030cc900d...,1200,2019-09-12 23:46:27.779,antidote,...,NaT,NaT,NaN,NaN,74210225.0,NaT,1girl :o arms_up bangs blue_dress blue_eyes bl...,3,A chat between a user and an assistant. The as...,"A young woman with blue eyes and hair, dressed..."
4,1girl black_gloves black_gothic_dress_(idolmas...,False,0.0,1,0,False,https://cdn.donmai.us/preview/27/ba/27baeb14fb...,586,2021-09-20 08:47:58.364,hiiragi_hazime,...,NaT,NaT,NaN,NaN,56566472.0,NaT,1girl black_gloves breasts brown_hair dress gl...,3,A chat between a user and an assistant. The as...,A female singer wearing black gloves and a dre...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,1girl animal_ears furry furry_female green_eye...,False,0.0,3,0,False,https://cdn.donmai.us/preview/a4/d8/a4d84e30af...,800,2021-08-28 18:02:24.851,koishi_chikasa,...,NaT,NaT,NaN,NaN,15198167.0,NaT,1girl animal_ears furry furry_female green_eye...,3,A chat between a user and an assistant. The as...,An anthropomorphic female character with green...
29996,1girl arm_up armpits bow breasts brown_hair ha...,False,0.0,1,0,False,https://cdn.donmai.us/preview/e8/35/e8355965a2...,703,2021-08-23 05:17:10.332,nannacy7,...,NaT,NaT,NaN,NaN,47359125.0,NaT,1girl arm_up armpits bow breasts brown_hair ha...,0,Reply with an eloquent caption based on all th...,"In the simplicity of the background, nannacy7 ..."
29997,1girl :< alternate_eye_color arm_up armpits ba...,False,1.0,2,0,False,https://cdn.donmai.us/preview/21/d0/21d0b9d805...,1000,2021-09-06 08:09:00.197,anmi,...,NaT,NaT,NaN,NaN,NaN,NaT,1girl :< alternate_eye_color arm_up armpits ba...,0,Reply with an eloquent caption based on all th...,With her sturgeon face and crescent hair ornam...
29998,1boy 2girls 4koma :d ahoge blonde_hair blue_ey...,False,1.0,4,0,False,https://cdn.donmai.us/preview/a2/7c/a27c781946...,1075,2021-07-26 20:50:42.519,otsu_kinoto,...,2017-02-27 14:05:14.208,2017-02-27 14:05:14.208,NaN,13793.0,43242731.0,NaT,1boy 2girls 4koma :d ahoge blonde_hair blue_ey...,0,Reply with an eloquent caption based on all th...,"""An enchanting 4koma featuring Kawashiro Nitor..."


In [46]:
data = load_dataset('json',data_files='data/datas/text_only.json')
data

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to C:/Users/panta/.cache/huggingface/datasets/json/default-0d2c8a509cc538f6/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['caption_string', 'filtered_tag_string'],
        num_rows: 30000
    })
})

In [47]:
BASE_MODEL = "microsoft/deberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
mlb = preprocessing.MultiLabelBinarizer(classes=tag_list)
mlb.fit([list(tag_list)])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
c:\Users\panta\anaconda3\envs\nlp\lib\site-packages\transformers\convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


MultiLabelBinarizer(classes=['1girl', 'solo', 'long_hair', 'breasts', 'blush',
                             'looking_at_viewer', 'smile', 'short_hair',
                             'open_mouth', 'bangs', 'blue_eyes',
                             'multiple_girls', 'blonde_hair', 'skirt',
                             'brown_hair', 'large_breasts', 'simple_background',
                             'black_hair', 'eyebrows_visible_through_hair',
                             'thighhighs', 'hair_ornament', 'hat', 'red_eyes',
                             'gloves', 'shirt', '1boy', 'dress',
                             'white_background', 'ribbon', 'long_sleeves', ...])

In [12]:
def bruh(data_point):
    result = tokenizer(
        data_point["caption_string"],
        truncation=True,
        max_length=512,
        padding=True,
    )
    data_point.update(result)

    tags = data_point["filtered_tag_string"].split(" ")
    data_point["tags"] = tags

    data_point["labels"] = torch.from_numpy(mlb.transform([tags])[0]).to(torch.float16)

    return data_point

bruh(data[0])['labels'].shape


torch.Size([6474])

In [48]:
data = data.map(bruh)
data

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['caption_string', 'filtered_tag_string', 'input_ids', 'token_type_ids', 'attention_mask', 'tags', 'labels'],
        num_rows: 30000
    })
})

In [49]:
train_testvalid = data['train'].train_test_split(0.10)
train_testvalid

DatasetDict({
    train: Dataset({
        features: ['caption_string', 'filtered_tag_string', 'input_ids', 'token_type_ids', 'attention_mask', 'tags', 'labels'],
        num_rows: 27000
    })
    test: Dataset({
        features: ['caption_string', 'filtered_tag_string', 'input_ids', 'token_type_ids', 'attention_mask', 'tags', 'labels'],
        num_rows: 3000
    })
})

In [50]:
test_valid = train_testvalid['test'].train_test_split(0.5)
test_valid

DatasetDict({
    train: Dataset({
        features: ['caption_string', 'filtered_tag_string', 'input_ids', 'token_type_ids', 'attention_mask', 'tags', 'labels'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['caption_string', 'filtered_tag_string', 'input_ids', 'token_type_ids', 'attention_mask', 'tags', 'labels'],
        num_rows: 1500
    })
})

In [51]:
train_test_valid_dataset = datasets.DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})
train_test_valid_dataset

DatasetDict({
    train: Dataset({
        features: ['caption_string', 'filtered_tag_string', 'input_ids', 'token_type_ids', 'attention_mask', 'tags', 'labels'],
        num_rows: 27000
    })
    test: Dataset({
        features: ['caption_string', 'filtered_tag_string', 'input_ids', 'token_type_ids', 'attention_mask', 'tags', 'labels'],
        num_rows: 1500
    })
    valid: Dataset({
        features: ['caption_string', 'filtered_tag_string', 'input_ids', 'token_type_ids', 'attention_mask', 'tags', 'labels'],
        num_rows: 1500
    })
})

In [35]:
id2label = {k: v for k, v in enumerate(tag_list)}
label2id = {v: k for k, v in enumerate(tag_list)}
model = AutoModelForSequenceClassification.from_pretrained(
    BASE_MODEL,
    # torch_dtype=torch.float16,
    # device_map="auto",
    problem_type="multi_label_classification",
    num_labels=len(tag_list),
    id2label=id2label,
    label2id=label2id,
).to("cuda")

Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2ForSequenceClassification: ['lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.LayerNorm.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.bias', 'mask_predictions.dense.bias']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a

In [53]:
tag_counts = pd.read_json(r'data/dictionaries/tag_counts.json',lines=True)
tag_counts

,name,post_count
0,1girl,20971
1,solo,18612
2,long_hair,16075
3,looking_at_viewer,12860
4,smile,11850
...,...,...
6050,thigh_grab,1
6051,cursor,1
6052,knight_(chess),1
6053,backboob,1


In [54]:
from sklearn.metrics import *

def compute_metrics(p):
    pred, labels = p
    predictions = (pred > 0.1).astype(int)
    accuracy = accuracy_score(y_true=labels, y_pred=predictions)
    recall = recall_score(
        y_true=labels, y_pred=predictions, average="samples", zero_division=1
    )
    precision = precision_score(
        y_true=labels, y_pred=predictions, average="samples", zero_division=1
    )
    f1_samples = f1_score(
        y_true=labels, y_pred=predictions, average="samples", zero_division=1
    )
    f2_samples = fbeta_score(
        y_true=labels, y_pred=predictions, average="samples", zero_division=1, beta=2
    )
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_samples": f1_samples,
        "f2_samples": f2_samples,
    }

In [8]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    eval_dataset=data["test"],
    tokenizer=tokenizer,
    callbacks = [transformers.EarlyStoppingCallback(early_stopping_patience=3)],
    compute_metrics=compute_metrics,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=32,
        num_train_epochs=3,
        learning_rate=3e-4,
        logging_steps=5,
        optim="adamw_torch",
        evaluation_strategy="steps",
        eval_steps=20,
        save_strategy="steps",
        save_steps=40,
        save_total_limit=5,
        load_best_model_at_end=True,
        metric_for_best_model='f2_samples',
        greater_is_better=True,
        output_dir="training/outputs",
        report_to="tensorboard",
    ),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

In [9]:
trainer.train()

  0%|          | 0/444 [00:00<?, ?it/s]

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.6519, 'learning_rate': 0.0002966216216216216, 'epoch': 0.03}
{'loss': 0.4067, 'learning_rate': 0.0002932432432432432, 'epoch': 0.07}
{'loss': 0.1072, 'learning_rate': 0.00028986486486486487, 'epoch': 0.1}
{'loss': 0.0234, 'learning_rate': 0.0002864864864864865, 'epoch': 0.14}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 0.01568603515625, 'eval_accuracy': 0.0, 'eval_precision': 0.9849335213963547, 'eval_recall': 0.04036939313984169, 'eval_f1_micro': 0.07575194949870034, 'eval_f1_macro': 0.6051025156046363, 'eval_f1_weighted': 0.03095233770576623, 'eval_runtime': 21.0185, 'eval_samples_per_second': 47.482, 'eval_steps_per_second': 5.947, 'epoch': 0.14}
{'loss': 0.0156, 'learning_rate': 0.0002831081081081081, 'epoch': 0.17}
{'loss': 0.0157, 'learning_rate': 0.0002797297297297297, 'epoch': 0.2}
{'loss': 0.0156, 'learning_rate': 0.0002763513513513513, 'epoch': 0.24}
{'loss': 0.0141, 'learning_rate': 0.000272972972972973, 'epoch': 0.27}


...

  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 0.01245880126953125, 'eval_accuracy': 0.0, 'eval_precision': 0.9931464181661476, 'eval_recall': 0.023186015831134565, 'eval_f1_micro': 0.0448943099814803, 'eval_f1_macro': 0.6050422840112697, 'eval_f1_weighted': 0.019164925490050087, 'eval_runtime': 20.5675, 'eval_samples_per_second': 48.523, 'eval_steps_per_second': 6.078, 'epoch': 0.81}
{'loss': 0.0128, 'learning_rate': 0.0002155405405405405, 'epoch': 0.84}
{'loss': 0.013, 'learning_rate': 0.00021216216216216215, 'epoch': 0.88}
{'loss': 0.0124, 'learning_rate': 0.00020878378378378376, 'epoch': 0.91}
{'loss': 0.0123, 'learning_rate': 0.00020540540540540537, 'epoch': 0.95}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 0.01245880126953125, 'eval_accuracy': 0.0, 'eval_precision': 0.9851237765769005, 'eval_recall': 0.04224934036939314, 'eval_f1_micro': 0.07927961381359079, 'eval_f1_macro': 0.6051066884838372, 'eval_f1_weighted': 0.03314792242293518, 'eval_runtime': 19.9402, 'eval_samples_per_second': 50.05, 'eval_steps_per_second': 6.269, 'epoch': 0.95}
{'train_runtime': 722.7107, 'train_samples_per_second': 78.67, 'train_steps_per_second': 0.614, 'train_loss': 0.05372939620699201, 'epoch': 0.95}


TrainOutput(global_step=140, training_loss=0.05372939620699201, metrics={'train_runtime': 722.7107, 'train_samples_per_second': 78.67, 'train_steps_per_second': 0.614, 'train_loss': 0.05372939620699201, 'epoch': 0.95})

In [11]:
classifier = transformers.pipeline(
    "text-classification",
    model=model,
    device="cuda:0",
    tokenizer=tokenizer,
    return_all_scores=True,
)
prediction = classifier("a single man in a black suit")
prediction

[[{'label': '1girl', 'score': 0.7220050096511841},
  {'label': 'solo', 'score': 0.5797643661499023},
  {'label': 'long_hair', 'score': 0.5472673177719116},
  {'label': 'breasts', 'score': 0.3895508348941803},
  {'label': 'blush', 'score': 0.34181201457977295},
  {'label': 'looking_at_viewer', 'score': 0.33695554733276367},
  {'label': 'smile', 'score': 0.3286130726337433},
  {'label': 'short_hair', 'score': 0.27758362889289856},
  {'label': 'open_mouth', 'score': 0.2187088131904602},
  {'label': 'bangs', 'score': 0.22223761677742004},
  {'label': 'blue_eyes', 'score': 0.20454467833042145},
  {'label': 'multiple_girls', 'score': 0.16697928309440613},
  {'label': 'blonde_hair', 'score': 0.17063476145267487},
  {'label': 'skirt', 'score': 0.15179024636745453},
  {'label': 'brown_hair', 'score': 0.16054077446460724},
  {'label': 'large_breasts', 'score': 0.124974824488163},
  {'label': 'simple_background', 'score': 0.10988301038742065},
  {'label': 'black_hair', 'score': 0.1705668121576309

In [14]:
pred_tags = [x['label'] for x in prediction[0] if x['score'] > 0.15]
pred_tags

['1girl',
 'solo',
 'long_hair',
 'breasts',
 'blush',
 'looking_at_viewer',
 'smile',
 'short_hair',
 'open_mouth',
 'bangs',
 'blue_eyes',
 'multiple_girls',
 'blonde_hair',
 'skirt',
 'brown_hair',
 'black_hair',
 'eyebrows_visible_through_hair',
 'hat',
 'shirt']

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model.push_to_hub("ooferdoodles/text2tags-opt-350m", use_auth_token=True)

In [76]:
model.save_pretrained(r"classifiers/deberta-v3-base")

In [ ]:
%load_ext tensorboard